<a href="https://colab.research.google.com/github/owhat02/Crime_Prediction/blob/yolo8/yoloV8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!nvidia-smi

Fri Dec  6 06:38:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   76C    P0              34W /  72W |    623MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 32.7/235.7 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
from ultralytics import YOLO
import cv2
import os
from tqdm import tqdm

# person 모델 로드 (기본 COCO 모델)
person_model = YOLO('yolov8s.pt')  # COCO 데이터셋으로 학습된 기본 YOLO 모델

# knife 모델 로드 (커스텀 학습된 모델)
knife_model = YOLO('/content/best.pt')  # knife 학습된 모델

# 커스텀 라벨 정의
custom_labels = {0: "Person", 1: "Knife"}

def detect_objects(frame):
    # person 모델로 예측
    person_results = person_model.predict(source=frame, conf=0.35, classes=[0])

    # knife 모델로 예측
    knife_results = knife_model.predict(source=frame, conf=0.35, classes=[0])

    # person 탐지 결과
    person_boxes = person_results[0].boxes.xyxy.tolist()
    person_labels = [custom_labels[0]] * len(person_boxes)
    person_scores = [box.conf.item() for box in person_results[0].boxes]

    # knife 탐지 결과
    knife_boxes = knife_results[0].boxes.xyxy.tolist()
    knife_labels = [custom_labels[1]] * len(knife_boxes)
    knife_scores = [box.conf.item() for box in knife_results[0].boxes]

    # 결과 결합
    all_boxes = person_boxes + knife_boxes
    all_labels = person_labels + knife_labels
    all_scores = person_scores + knife_scores

    return all_boxes, all_labels, all_scores

def annotate_frame(frame, boxes, labels, scores):
    # 탐지 결과를 프레임에 그리기
    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = map(int, box)
        color = (0, 255, 0) if label == "Person" else (0, 0, 255)  # 색상 설정
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label} {score:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

# 동영상 경로 설정
input_video_path = '/content/test_video/KakaoTalk_20241204_192440335.mp4'  # 입력 동영상 경로
output_video_path = '/content/output_video_192440335.mp4'  # 출력 동영상 경로

# 동영상 파일 읽기
cap = cv2.VideoCapture(input_video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 동영상 파일 쓰기 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# 프레임별 처리
for _ in tqdm(range(frame_count), desc="Processing Video"):
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 탐지
    boxes, labels, scores = detect_objects(frame)

    # 프레임에 시각화 적용
    annotated_frame = annotate_frame(frame, boxes, labels, scores)

    # 결과 프레임 저장
    video_writer.write(annotated_frame)

# 자원 해제
cap.release()
video_writer.release()

print(f"처리된 동영상이 {output_video_path}에 저장되었습니다.")


Processing Video:   0%|          | 0/257 [00:00<?, ?it/s]
0: 384x640 1 person, 7.9ms
Speed: 1.7ms preprocess, 7.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 knife, 8.0ms
Speed: 1.7ms preprocess, 8.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Processing Video:   0%|          | 1/257 [00:00<01:44,  2.45it/s]
0: 384x640 1 person, 7.6ms
Speed: 1.6ms preprocess, 7.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 knife, 7.4ms
Speed: 1.5ms preprocess, 7.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8.3ms
Speed: 1.6ms preprocess, 8.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 knife, 7.4ms
Speed: 1.6ms preprocess, 7.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.6ms
Speed: 1.7ms preprocess, 7.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1

처리된 동영상이 /content/output_video_192440335.mp4에 저장되었습니다.
